In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print("ix: ", ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[67, 58,  1, 59, 68, 71,  1, 72],
        [56, 64,  9,  3,  1, 72, 61, 58],
        [ 1, 37, 54, 60, 62, 56,  1, 26],
        [65, 65, 59, 74, 65, 65, 78,  1]], device='cuda:0')
targets:
tensor([[58,  1, 59, 68, 71,  1, 72, 58],
        [64,  9,  3,  1, 72, 61, 58,  1],
        [37, 54, 60, 62, 56,  1, 26, 58],
        [65, 59, 74, 65, 65, 78,  1, 74]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


*kv(B﻿e]OG2-Ey_f2&Czy_4Gcq6B﻿j2d*MWV*Gmfh_zvnTtiYs'&**X(r3,vb(Y[Z xlX]dI
gV5mdAcK1&r l-EZ.vMUjV" ,Y-J3Q(h2ZgA:D
"7﻿DV*K]d
mj3b﻿Ow?(N!fj[!?Bl*-OK&?LU
l'yFNIN7BKtfj9)h&m]pScrK)rMf*f6-(K9mEJw42tLRs8!wvNd5NlTz9*p
06ptiqyE,rL1MYwY8qfe.2BwDFBw42u﻿1N! [L]p

0a1
Hi)f0eu5wHtGD:Zg"C!G22KLt!zp
LY-Mcd73f12tiV'1C8 dBxp
E6"﻿mj1v5PC0OW﻿Dk7P-swQa1 jAHX dK2S1Y*vb&xysHi4Y]*G7ETdn"Rs3-
zHnDc bKM8m3uYwmnbK'lGlTs.uu6&K6e:e]_Nd!Cja) *ffXMp1_QxC!1XU9kZBlTTnQq6,z,':!wDV7IedCu(EBfoYDr vb_t2K kZLrTe73xcaH2g﻿,r Yb5);7btm(


In [14]:
# PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, training loss: {losses['train']:.3f}, validation loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, training loss: 2.481, validation loss: 2.553
step: 250, training loss: 2.515, validation loss: 2.535
step: 500, training loss: 2.495, validation loss: 2.517
step: 750, training loss: 2.483, validation loss: 2.530
2.755837917327881


ReLU(inplace=True)
